In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm
import json
from datasets import load_dataset

In [2]:
MODEL_DIR = "models/4096_8_-1/"
NUM_POS, NUM_NEG = 5, 5 

In [3]:
RNG = np.random.default_rng(seed=910)

In [1]:
df = pd.DataFrame(load_dataset(DATA_PATH, split="train"))
all_sparse_embeddings = sparse.load_npz(f"{MODEL_DIR}/sparse_embeddings.npz")

assert df.shape[0] == all_sparse_embeddings.shape[0]

NameError: name 'pd' is not defined

In [ ]:
def get_positive_and_negative_samples(all_sparse_embeddings, df, feature_idx, num_pos=5, num_neg=5):
    feature_column = all_sparse_embeddings[:, [feature_idx]]
    
    dense_feature = feature_column.toarray().flatten()
    
    non_zero_indices = np.nonzero(dense_feature)[0]
    
    sorted_indices = non_zero_indices[np.argsort(dense_feature[non_zero_indices])[::-1]]
    
    max_activating = sorted_indices[:num_pos]
    
    zero_activating = np.where(dense_feature == 0)[0]
    
    zero_activating = np.random.choice(zero_activating, size=num_neg, replace=False)
    
    max_activating_sentences = df.iloc[max_activating]["sentence"].tolist()
    max_activating_activations = dense_feature[max_activating]
    zero_activating_sentences = df.iloc[zero_activating]["sentence"].tolist()
    
    return max_activating_sentences, max_activating_activations, zero_activating_sentences

In [6]:
def format_sentences(max_sents, max_acts, zero_sents):
    max_formatted = "\n"
    zero_formatted = "\n"
    for i, (msent, act) in enumerate(zip(max_sents, max_acts)):
        max_formatted += f"EXEMPLE {i+1}. {msent}\n ACTIVATION: {act}\n" 
    for j, zsent in enumerate(zero_sents):
        zero_formatted += f"EXEMPLE {j+1}. {zsent}\n"

    return max_formatted, zero_formatted

In [7]:
with open("interpreter_prompt_fr_v2.txt", "r") as f:
    base_prompt = f.read()

num_features = all_sparse_embeddings.shape[1]

interpreter_prompts = {}

for feature in tqdm(range(num_features)):
    max_sents, max_acts, zero_sents = get_positive_and_negative_samples(
        all_sparse_embeddings, 
        df, 
        feature, 
        num_pos=NUM_POS,
        num_neg=NUM_NEG
    )
    max_formatted, zero_formatted = format_sentences(max_sents, max_acts, zero_sents)
    formatted_prompt = base_prompt.format(max_activating=max_formatted, zero_activating=zero_formatted)
    interpreter_prompts[feature] = formatted_prompt

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4096/4096 [00:20<00:00, 203.20it/s]


In [8]:
with open(f"{MODEL_DIR}/interpreter_prompts_v2.json", "w") as f: 
    json.dump(interpreter_prompts, f)